In [2]:
import pandas as pd
import plotly.express as px
import regex as re

## Import pre-processed data

In [3]:
df = pd.read_pickle("US_Accidents_Processed_df.pkl")
original_df = df

In [4]:
findings_ls = list(set(df['finding_description']))
# # search unique list
# for finding in findings_ls:
#     if 'power plant' in finding:
#         print(finding)
#     else:
#         continue

In [5]:
# Check only the values with the specific fault type - in this case choose 'Aircraft'
# Create a new df with only this data
# Create a new column with the system names - structures, propeller/rotor etc.
# count the number of times they occur#
# plot the data in a pie chart

In [6]:
len(df)

173438

In [7]:
df = df[df['finding_description'].str.startswith('A')] 

In [8]:
new_findings_ls = list(set(df['finding_description']))

In [10]:
system_ls = []
for sys in new_findings_ls:
    ls = re.split('-', sys)
    system_ls.append(ls[2])
system_ls = list(set(system_ls))
def find_type(description): #description is a place holder for whatever column the function is applied to
    for type in system_ls:
        if type in description:
            return type
    return None
# Apply the function to create a new column
df['system_type'] = df['finding_description'].apply(find_type)

C:\Users\yxb19198\AppData\Local\Temp\ipykernel_26004\2764924611.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['system_type'] = df['finding_description'].apply(find_type)


In [11]:
df.head(5)

,ev_id,Aircraft_Key_x,ntsb_no,acft_missing,acft_make,acft_model,acft_series,acft_category,homebuilt,fixed_retractable,...,Aircraft_Key_y,eng_type,eng_mfgr,eng_model,power_units,finding_code,finding_description,Occurrence_Code,Occurrence_Description,system_type
15,20130919X02109,1,ENG13WA040,N,SIKORSKY,S76 - A,A,HELI,N,RETR,...,1,TS,Rolls-Royce,250-C30,650.0,105723002,Aircraft-Aircraft power plant-Engine (turbine/...,252150,Taxi-into takeoff position Fire/smoke (non-imp...,Engine (turbine/turboprop)
16,20130919X02109,1,ENG13WA040,N,SIKORSKY,S76 - A,A,HELI,N,RETR,...,1,TS,Rolls-Royce,250-C30,650.0,105723002,Aircraft-Aircraft power plant-Engine (turbine/...,253150,Taxi-from runway Fire/smoke (non-impact),Engine (turbine/turboprop)
17,20130919X02109,1,ENG13WA040,N,SIKORSKY,S76 - A,A,HELI,N,RETR,...,1,TS,Rolls-Royce,250-C30,650.0,105723002,Aircraft-Aircraft power plant-Engine (turbine/...,153150,Standing-engine(s) operating Fire/smoke (non-i...,Engine (turbine/turboprop)
18,20130919X02109,1,ENG13WA040,N,SIKORSKY,S76 - A,A,HELI,N,RETR,...,1,TS,Rolls-Royce,250-C30,650.0,105723002,Aircraft-Aircraft power plant-Engine (turbine/...,252150,Taxi-into takeoff position Fire/smoke (non-imp...,Engine (turbine/turboprop)
19,20130919X02109,1,ENG13WA040,N,SIKORSKY,S76 - A,A,HELI,N,RETR,...,1,TS,Rolls-Royce,250-C30,650.0,105723002,Aircraft-Aircraft power plant-Engine (turbine/...,253150,Taxi-from runway Fire/smoke (non-impact),Engine (turbine/turboprop)


# Analyse the final result for severity ranking of overall initial faults
Broken down into mission phase - description

Not all of the initial mission phases are seperated via dash 

In [19]:
result_ls = list(set(df['Occurrence_Description']))
result_ls

['Landing-flare/touchdown Sys/Comp malf/fail (non-power)',
 'Landing-flare/touchdown Loss of control in flight',
 'Landing-flare/touchdown Miscellaneous/other',
 'Enroute-climb to cruise Flight instrument malf/fail',
 'Unknown Aerodynamic stall/spin',
 'Enroute-descent Cabin safety event',
 'Approach-IFR final approach Fuel starvation',
 'Enroute-cruise Fuel starvation',
 'Landing-aborted after touchdown Loss of engine power (total)',
 'Landing Runway excursion',
 'Takeoff Aerodynamic stall/spin',
 'Standing-engine(s) start-up AC/prop/rotor contact w person',
 'Maneuvering-hover Miscellaneous/other',
 'Enroute-climb to cruise Off-field or emergency landing',
 'Maneuvering-aerobatics Low altitude operation/event',
 'Landing-landing roll Collision during takeoff/land',
 'Standing-engine(s) operating Preflight or dispatch event',
 'Approach Flight control sys malf/fail',
 'Enroute-descent Loss of visual reference',
 'Approach-VFR pattern final Off-field or emergency landing',
 'Maneuverin

In [62]:
for result in result_ls:
    if 'Post' in result:
        print(result)

Post-impact Evacuation
Post-impact Cabin safety event
Post-impact Flight control sys malf/fail
Post-impact Dynamic rollover
Post-impact Fire/smoke (post-impact)
Post-impact Landing gear collapse
Post-impact Explosion (post-impact)
Post-impact Aircraft structural failure
Post-impact Miscellaneous/other
Post-impact Part(s) separation from AC
Post-impact Sys/Comp malf/fail (non-power)
Post-impact Ground collision
Post-impact Attempted remediation/recovery
Post-impact Hazardous material leak/spill
Post-impact Roll over
Post-impact Collision with terr/obj (non-CFIT)


In [25]:
re.split('-', result_ls[4])

['Unknown Aerodynamic stall/spin']

In [38]:
re.findall('[A-Z][^A-Z]*', result_ls[4])

['Unknown ', 'Aerodynamic stall/spin']

In [26]:
len(result_ls)

1228

In [60]:
#Preprocess the data to select the variables before a dash 
mission_phase_ls = []
mission_phase_ls_v2 = []
for result in result_ls:
    word_ls = re.split('-', result)
    if len(word_ls) == 1:
        split_word_ls = re.findall('[A-Z][^A-Z]*', word_ls[0]) # doesn't fully work on specific words so need to check again?
        mission_phase_ls.append(split_word_ls[0].rstrip())
    else:
        mission_phase_ls.append(word_ls[0].rstrip()) # need to account for words where the dash is part of the word so need to check if it's capital after the dash
mission_phase_ls = list(set(mission_phase_ls))
# for now my double check
for mis in mission_phase_ls:
    split_word_ls = re.findall('[A-Z][^A-Z]*', mis)
    mission_phase_ls_v2.append(split_word_ls[0].rstrip())
mission_phase_ls = list(set(mission_phase_ls_v2))
#Then for the remaining information, select the words until a capital letter is encountered
#Therefore have a collection of data based on the first condition and then create a similar setup for diagrams for the severity level

In [61]:
mission_phase_ls

['Pushback/towing',
 'Pushback/tow',
 'After landing',
 'Maneuvering',
 'Uncontrolled descent',
 'Unknown',
 'Enroute',
 'Prior to flight',
 'Other',
 'Post',
 'Taxi',
 'Initial climb',
 'Autorotation',
 'Standing',
 'Landing',
 'Takeoff',
 'Approach',
 'Emergency descent']

In [39]:
re.findall('[A-Z][^A-Z]*', mission_phase_ls[4])

['Prior to flight ', 'Fire/smoke (non']

# Fault Frequencies

In [ ]:
fault_freq = []
fault_dict = {}
for system in system_ls:
    freq = df['system_type'].value_counts()[system]
    fault_freq.append(int(freq))
    fault_dict[system] = int(freq) # convert from np.int64 into int representation

In [ ]:
df[df['system_type'] == 'Misc hardware']

In [ ]:
df['system_type'].value_counts()['Misc hardware']

In [ ]:
fig = px.pie(values=fault_freq, names=system_ls, title='Frequency of aircraft faults')
fig.show()

In [ ]:
print(fault_dict)

## Old code

In [ ]:
# Identify the aircraft faults and their frequency
# Start by only choosing faults with 'aircraft' type
ac_ls = []
for finding in findings_ls:
    if 'Aircraft' in re.split('-', finding)[0]:
        ac_ls.append(finding)

In [ ]:
len(ac_ls)

In [ ]:
# Then create a pie chart of the frequency of the faults as part of the whole
ac_fault_freq = []
for ac_fault in ac_ls:
    freq = df['finding_description'].value_counts()[ac_fault]
    ac_fault_freq.append(freq)
ac_fault_freq = [int(x) for x in ac_fault_freq] # convert from np.int64 into int representation

In [ ]:
# remove faults that occur less than 20 times
ac_fault_names = []
ac_fault_freq_ver1 = []
for i in range(len(ac_fault_freq)):
    if ac_fault_freq[i] < 200:
        continue
    else:
        ac_fault_names.append(ac_ls[i])
        ac_fault_freq_ver1.append(ac_fault_freq[i])
print(len(ac_fault_names))

In [ ]:
fig = px.pie(values=ac_fault_freq_ver1, names=ac_fault_names, title='Frequency of aircraft faults')
fig.show()

In [ ]:
# How to group faults based on the overall subsystem and count them based on that category instead?
# Create a new column based on each finding index of interest (the general subsystems)
# create new column with only the types of faults so I can identify how many of each have ocurred
type_ls = []
for finding in findings_ls:
    ls = re.split('-', finding)
    type_ls.append(ls[0])
type_ls = list(set(type_ls))
type_ls
def find_type(description): #description is a place holder for whatever column the function is applied to
    for type in type_ls:
        if type in description:
            return type
    return None
# Apply the function to create a new column
df['finding_type'] = df['finding_description'].apply(find_type)

In [ ]:
print(type_ls)

### Figure out how to make one function to call instead of copy and pasting same fucntion and changing the list called in the function

In [ ]:
system_ls = []
for sys in findings_ls:
    ls = re.split('-', sys)
    system_ls.append(ls[1])
system_ls = list(set(system_ls))
def find_type(description): #description is a place holder for whatever column the function is applied to
    for type in system_ls:
        if type in description:
            return type
    return None
# Apply the function to create a new column
df['system_type'] = df['finding_description'].apply(find_type)

In [ ]:
system_ls

In [ ]:
# count the number of system faults
sys_fault_freq = []
for system in system_ls:
    freq = df['system_type'].value_counts()[system]
    sys_fault_freq.append(freq)
sys_fault_freq = [int(x) for x in sys_fault_freq] # convert from np.int64 into int representation
print(sys_fault_freq)

## Analyse data specific to helicopters

In [ ]:
RC_df = df.loc[df['acft_category'] == 'HELI']
RC_df.head(5)

In [ ]:
# Create a pie chart of the frequency of certain findings codes 
# Initially make a dictionary with codes to the number of times it has ocurred
Occur_ls = list(set(RC_df.loc[:, 'Occurrence_Code']))
Occur_dict = {}
Other_occur_ls = []
for occur in Occur_ls:
    if len(RC_df.loc[RC_df['Occurrence_Code'] == occur]) > 0:
        occur_name = list(RC_df.loc[RC_df['Occurrence_Code'] == occur]['Occurrence_Description'])[0]
        Occur_dict[occur_name] = len(RC_df.loc[RC_df['Occurrence_Code'] == occur])
    else:
        Other_occur_ls.append(occur) # What codes never occur in helicopters?
# Extract only the relevant codes for plotting

In [ ]:
# Sort through dictionary and only include the more frequent code events
Occur_dict = {k: v for k, v in Occur_dict.items() if v > 100}
Occur_vals = list(Occur_dict.values())
Occur_keys = list(Occur_dict.keys())

In [ ]:
# df.loc[df['pop'] < 2.e6, 'country'] = 'Other countries' # Represent only very frequent incidents
fig = px.pie(values=Occur_vals, names=Occur_keys, title='Frequency of types of landings in Helicopters')
fig.show()
# show the full names of the values with the highest values?

In [ ]:
Occur_ls = list(set(df.loc[:, 'finding_code']))
Occur_dict = {}
Other_occur_ls = []
for occur in Occur_ls:
    if len(RC_df.loc[RC_df['finding_code'] == occur]) > 0:
        occur_name = list(RC_df.loc[RC_df['finding_code'] == occur]['finding_description'])[0]
        Occur_dict[occur_name] = len(RC_df.loc[RC_df['finding_code'] == occur])
    else:
        Other_occur_ls.append(occur) # What codes never occur in helicopters?
# Extract only the relevant codes for plotting

In [ ]:
# Plot the frequencies on a bar plot
fig = px.box(RC_df, y=Occur_vals, log_y=True)
fig.show()

In [ ]:
# Sort through dictionary and only include the more frequent code events
Occur_dict = {k: v for k, v in Occur_dict.items() if v > 100}
Occur_vals = list(Occur_dict.values())
Occur_keys = list(Occur_dict.keys())
Occur_keys

In [ ]:
fig = px.pie(values=Occur_vals, names=Occur_keys, title='Frequency of Accident Types in Helicopters')
fig.show()

In [ ]:
# Figure out how many accidents occur based on group type (string before first '-') in findings
# populate column with only the first term in occur_keys - if string in other string then select that string
type_ls = []
for key in Occur_keys:
    ls = re.split('-', key)
    type_ls.append(ls[0])
type_ls = list(set(type_ls))
type_ls

In [ ]:
# create new column with only the types of faults so I can identify how many of each have ocurred
def find_type(description): #description is a place holder for whatever column the function is applied to
    for type in type_ls:
        if type in description:
            return type
    return None
# Apply the function to create a new column
RC_df['finding_type'] = RC_df['finding_description'].apply(find_type)

In [ ]:
# Figure out values for each type of fault and how many times they have occurred
len_type_ls = []
for type in type_ls:
    count = len(RC_df[RC_df['finding_type'] == type])
    len_type_ls.append(count)
# create a pie chart
fig = px.pie(values=len_type_ls, names=type_ls, title='Frequency of Types of Accidents')
fig.show()

In [ ]:
# Limit the accidents to only aircraft specific ones
RC_df_air = RC_df[RC_df['finding_type'] == 'Aircraft']
# Highlight the affected subsystem (usually the 3rd and 4th is most relevant)
# will start with the 3rd using the same method as previously except a lot more options now
# create a list of all possible options
air_fault_ls = []
for key in Occur_keys:
    ls = re.split('-', key)
    val = ls[3] 
    air_fault_ls.append(val)
air_fault_ls = list(set(air_fault_ls))
air_fault_ls

In [ ]:
def find_component_type(description):
    for air_fault in air_fault_ls:
        if air_fault in description:
            return air_fault
    return None
# Apply the function to create a new column
RC_df['aircraft_type_finding'] = RC_df['finding_description'].apply(find_component_type)

In [ ]:
len_air_fault_ls = []
for type in air_fault_ls:
    count = len(RC_df[RC_df['aircraft_type_finding'] == type])
    len_air_fault_ls.append(count)
print(len_air_fault_ls)
# create a pie chart
fig = px.pie(values=len_air_fault_ls, names=air_fault_ls, title='Frequency of Types of Accidents (Aircraft type)')
fig.show()
# fig.write_image("images/fig1.png")